In [ ]:
# Install selenium itself for the first time
!pip install selenium

In [1]:
# Install libraries dependencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
from bs4 import BeautifulSoup
import json
import os
import re
from selenium.webdriver.firefox.options import Options
import csv

In [ ]:
# Retrieve Answers to Questions within CSV file
def retrieve(things):
    topic = things[0][0]
    subdirectory = "Machine Learning" + " GPT Answer"
    filename = re.sub(r'[<>:"/\\|?*]', '', topic) + '.jsonl'
    filename = os.path.join(subdirectory, filename)
    # Creates a temp file to store all the questions that are already done
    temp = []
    with open(filename, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            line = json.loads(line)
            temp.append(line["instruction"])
    options = Options()
    # CHANGE THE LINK TO YOUR FIREFOX PROFILE WHERE CHATGPT IS LOGGED IN + ENABLE COOKIES
    options.profile = ''
    driver = webdriver.Firefox(options=options)
    driver.get("https://chat.openai.com/")
    time.sleep(7)
    # To iterate through all the questions listed in the question.csv file
    for i in things:
        topic = i[0]
        question = i[1]
        subdirectory = "Machine Learning" + " GPT Answer"
        #check if Subdirectory exists
        if not os.path.exists(subdirectory):
            os.makedirs(subdirectory)
        # The file is saved within a parent directory that is labelled as your topic name with each file labelled as the TOPIC name
        filename = re.sub(r'[<>:"/\\|?*]', '', topic) + '.jsonl'
        filename = os.path.join(subdirectory, filename)
        # Check if the row is empty in the CSV File or if answer was already inserted
        if len(i[0]) == 0 or question in temp:
            print(question + " Exists skipping...")
        else:
            # Creates a csv file and iterates between each answer, depositing it in a dictionary and appending it into the jsonl file
            with open(filename, 'a', encoding='utf-8') as json_file:
                print(question + " Does not exists, GPTing... ")
                # Select how many times you want to scroll down the web page to retrieve the questions
                for i in range(1):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                    time.sleep(2)
                html_source = driver.page_source
                soup_3 = BeautifulSoup(html_source, 'html.parser')
                # Input question using xpathh
                xpathh = "//textarea[contains(@tabindex,'0')]"
                text_input = driver.find_element(By.XPATH, value=xpathh)
                ActionChains(driver)\
                    .send_keys_to_element(text_input, question)\
                    .perform()
                # Clicking submit
                xpathh = "//button[contains(@class,'absolute bottom-1.5 right-2 rounded-lg border border-black bg-black p-0.5 text-white transition-colors enabled:bg-black disabled:text-gray-400 disabled:opacity-10 dark:border-white dark:bg-white dark:hover:bg-white md:bottom-3 md:right-3')]"
                clickable = driver.find_element(By.XPATH, xpathh)
                ActionChains(driver)\
                    .click(clickable)\
                    .perform()
                # Increase time if cutting off
                time.sleep(20)
                # Find all answer using xpath
                xpathh = "(//div[contains(@class,'markdown prose w-full break-words dark:prose-invert light')])"
                elements = driver.find_elements(By.XPATH, value=xpathh)
                # Since find elements returns an array, we only take the first index (Since there is only 1 answer)
                element = elements[0]
                inner_html = element.get_attribute('innerHTML')
                soup = BeautifulSoup(inner_html, 'html.parser')
                question_text = soup.get_text(separator=' ', strip=True)
                # Create a dictionary for the question and answer pair
                qa_dict = {"instruction" : question, "context" : "" , "responses" : question_text}
                json_file.write(json.dumps(qa_dict, ensure_ascii=False) + '\n')
                # Clicking new chat to restart the whole thing again
                xpathh = "//button[contains(@class, 'text-token-text-primary')]"
                clickable = driver.find_element(By.XPATH, xpathh)
                ActionChains(driver)\
                    .click(clickable)\
                    .perform()
                time.sleep(1)
    driver.quit()




In [24]:
#Removes empty files due to error links within the subdirectory
def remove_empty_files(subdirectory):
    # Iterate over files in the subdirectory
    for filename in os.listdir(subdirectory):
        filepath = os.path.join(subdirectory, filename)
        # Check if the file is empty
        if os.path.isfile(filepath) and os.path.getsize(filepath) == 0:
            print("Removing empty file:", filename)
            os.remove(filepath)

In [25]:
#Reads the file and retrieves the list of questions along side with the topic
# Define the directory containing the files
directory = "topics"
# Get a list of all files in the directory
files = os.listdir(directory)
#Questions CSV is formatted per row as (TOPICS)
for file in files:
    if file.endswith(".csv"):
        filepath = os.path.join(directory, file)
        with open(filepath) as topics:
            next(topics)
            things = []
            for line in topics:
                line = line.rstrip("\n")
                if len(line) == 0:
                    continue
                else:
                    line = line.split(",")
                    topic = line[0]
                    question = line[1]
                    things.append([topic,question])
            retrieve(things)
            # Cleans the subdirectory if there are empty files due to empty question csv files
            subdirectory = "Machine Learning" + " GPT Answer"
            remove_empty_files(subdirectory)


What advantages does Nadam optimization offer over Adam optimization? Exists skipping...
What mathematical principles underlie the Adam optimization algorithm? Exists skipping...
What is the role of momentum in the Adam optimization algorithm? Exists skipping...
What impact does adaptive learning rates have on the performance of Adam and Nadam optimization? Exists skipping...
What is the computational complexity of the Nadam optimization algorithm compared to Adam? Exists skipping...
What strategies are employed in Adam and Nadam to deal with noisy gradients? Exists skipping...
What is the effect of the beta parameters in Adam and Nadam optimization? Exists skipping...
What are some practical scenarios where Nadam outperforms Adam in training neural networks? Exists skipping...
What are the limitations of using Adam and Nadam optimization algorithms? Exists skipping...
What are the similarities between Adam and Nadam optimization techniques? Exists skipping...
What insights from adapti

WebDriverException: Message: Failed to decode response from marionette


In [ ]:
# Test run
# This should create open a browser to search for this questions
retrieve([["Machine Learning", "What is Machine Learning"]])